#️⃣ CELL 1: 📘 Title + Overview (Markdown)
"""
# 🤖 Industrial Waste Compliance - Stacking Classifier Training

This notebook performs:
1. Data Loading & Preparation  
2. Feature Scaling & PCA  
3. Model Training (Stacking Classifier)  
4. Evaluation with Visualizations  
5. Scatter Plots for PCA and Prediction Insights 🎨

Let's dive in! 🚀
"""


In [ ]:
#️⃣ CELL 2: 🧩 Import Libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import joblib
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Aesthetic setup
sns.set(style="whitegrid", palette="cool")
plt.rcParams["figure.figsize"] = (8, 5)


In [ ]:
#️⃣ CELL 3: 📂 Load Dataset

os.makedirs("../models", exist_ok=True)

df = pd.read_csv("../data/cleaned_compliance_data.csv")
target_col = "Perfect_Waste_Decomposition_System"

X = df.drop(columns=[target_col])
y = df[target_col]

# Save feature columns for reuse
feature_columns = X.columns.tolist()
joblib.dump(feature_columns, '../models/feature_columns.pkl')

print(f"✅ Dataset Loaded: {df.shape[0]} rows × {df.shape[1]} columns")
display(df.head())


In [ ]:
#️⃣ CELL 4: ✂️ Train-Test Split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print("🔹 Training set shape:", X_train.shape)
print("🔸 Testing set shape:", X_test.shape)


In [ ]:
#️⃣ CELL 5: ⚖️ Feature Scaling

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

joblib.dump(scaler, "../models/scaler.pkl")
print("✅ Scaler saved successfully!")


In [ ]:
#️⃣ CELL 6: 🧠 PCA (Keep 95% Variance)

pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
joblib.dump(pca, "../models/pca.pkl")

print(f"✅ PCA Applied: Reduced from {X_train.shape[1]} → {X_train_pca.shape[1]} components")


In [ ]:
#️⃣ CELL 7: 🎨 PCA Scatter Visualization (2 Components)

if X_train_pca.shape[1] >= 2:
    plt.figure(figsize=(8,6))
    sns.scatterplot(
        x=X_train_pca[:, 0], 
        y=X_train_pca[:, 1],
        hue=y_train,
        palette="cool",
        alpha=0.7,
        edgecolor="k"
    )
    plt.title("🔹 PCA Feature Space (First 2 Components)")
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.legend(title="Target")
    plt.show()
else:
    print("⚠️ Not enough PCA components for scatter visualization.")


In [ ]:
#️⃣ CELL 8: ⚙️ Define Models for Stacking

base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('nb', GaussianNB())
]

meta_model = LogisticRegression(max_iter=1000, random_state=42)

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

print("✅ Base and Meta Models Defined Successfully!")


In [ ]:
#️⃣ CELL 9: 🧩 Train the Stacking Model

stack_model.fit(X_train_scaled, y_train)
print("🏋️‍♂️ Training Completed!")

# Save trained model
joblib.dump(stack_model, "../models/stack_model.pkl")
print("✅ Model saved at: ../models/stack_model.pkl")


In [ ]:
#️⃣ CELL 10: 📊 Evaluate Model Performance

y_pred = stack_model.predict(X_test_scaled)

print("🏆 **Stacking Model Performance (Scaled Data)**")
print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.title("Confusion Matrix - Stacking Model")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
#️⃣ CELL 11: 🌈 Scatter Plot - Prediction Results (True vs Predicted)

# Use PCA for 2D visualization
if X_test_pca.shape[1] >= 2:
    plt.figure(figsize=(8,6))
    sns.scatterplot(
        x=X_test_pca[:,0], 
        y=X_test_pca[:,1],
        hue=y_pred,
        style=y_test,
        palette="viridis",
        alpha=0.8
    )
    plt.title("🧩 Model Predictions in PCA Space")
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.legend(title="Predicted Label")
    plt.show()


In [ ]:
#️⃣ CELL 12: 🔥 Base Models Comparison

results = []
for name, model in base_models:
    model.fit(X_train_scaled, y_train)
    y_pred_base = model.predict(X_test_scaled)
    results.append({
        "Model": name.upper(),
        "Accuracy": accuracy_score(y_test, y_pred_base),
        "Precision": precision_score(y_test, y_pred_base, zero_division=0),
        "Recall": recall_score(y_test, y_pred_base, zero_division=0),
        "F1-Score": f1_score(y_test, y_pred_base, zero_division=0)
    })

results_df = pd.DataFrame(results).sort_values(by="F1-Score", ascending=False)
display(results_df)


In [ ]:
#️⃣ CELL 13: 📈 Visualization - Base Models Performance

plt.figure(figsize=(10, 6))
sns.heatmap(results_df.set_index("Model"), annot=True, fmt=".2f", cmap="YlGnBu")
plt.title("📊 Base Models Performance Comparison")
plt.ylabel("Model")
plt.show()

# Bar chart
plt.figure(figsize=(8,5))
sns.barplot(x="Model", y="Accuracy", data=results_df, palette="cool")
plt.title("Model Accuracy Comparison")
plt.ylim(0, 1)
plt.show()


#️⃣ CELL 14: ✅ Final Summary (Markdown)
"""
# ✅ Summary

- Dataset cleaned and standardized with **StandardScaler**  
- **PCA** reduced dimensionality to capture 95% variance  
- Built an ensemble **Stacking Classifier** with:
  - Random Forest 🌲
  - Decision Tree 🌳
  - KNN 👥
  - Naive Bayes 📈  
  - Logistic Regression ⚙️ (meta model)
- Visualized:
  - PCA space before training  
  - Prediction scatter in PCA 2D  
  - Confusion matrix  
  - Base model heatmap and accuracy chart  

🎯 Final trained model saved: `../models/stack_model.pkl`
"""
